<a href="https://colab.research.google.com/github/chuan1215/DLI_GroupAG/blob/main/TangSiChuon_KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount My Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')